In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# 武汉市
roi = ee.FeatureCollection('users/311605001111/ChinaCity').filter(ee.Filter.eq('市','武汉市'))
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,8)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

def area_after_clip(image):
    image_area = image.clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    areas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14
    })
    return image.set({'area': areas.get('B2')})

In [5]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(water_index)
Map.addLayer(landsat_images.mosaic(),visParams,'landsat image')

# 筛选得出采样影像

In [6]:
image_area = landsat_images.mosaic().select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14
})

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('area',ee.Number(areas.get('B2')).divide(10)))
dates = sample_image.aggregate_array('system:time_start').sort().map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

34
['2018-01-11', '2018-02-03', '2018-02-12', '2018-02-28', '2018-03-23', '2018-03-23', '2018-04-08', '2018-04-08', '2018-04-17', '2018-04-24', '2018-06-11', '2018-06-27', '2018-06-27', '2018-07-13', '2018-07-13', '2018-07-22', '2018-07-29', '2018-07-29', '2018-08-07', '2018-08-23', '2018-08-30', '2018-08-30', '2018-09-08', '2018-09-15', '2018-09-15', '2018-09-24', '2018-10-10', '2018-10-17', '2018-10-17', '2018-10-26', '2018-11-02', '2018-11-02', '2018-11-27', '2018-12-13']


# 多水体频率数据集的类别范围确定

In [7]:
# ndvi_mndwi水体频率
basemap = ee.Image.constant(0).clip(roi).rename('waterclass')
water_layer = landsat_images.map(lambda i : i.select('ndvi_mndwi').lt(0)).sum()
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
index_waterfrequency = water_layer.select('ndvi_mndwi').divide(validPixel_layer.select('count')).select('ndvi_mndwi').rename('frequency')
index_permanent = index_waterfrequency.gte(0.75).remap([0,1],[0,2]).rename('waterclass')
index_season= ee.ImageCollection([index_waterfrequency.gte(0.25),index_waterfrequency.lt(0.75)]).sum().eq(2).rename('waterclass')
index_waterclass = ee.ImageCollection([index_permanent,index_season,basemap]).sum()
# JRC
JRC_2018 = ee.Image("JRC/GSW1_3/YearlyHistory/2018").clip(roi).remap([0,1,2,3],[0,0,1,2]).rename('waterclass')
JRC_waterclass = ee.ImageCollection([JRC_2018,basemap]).sum()
# Maryland
Maryland_2018 = ee.Image('users/311605001111/Maryland/Maryland_2018').clip(roi).select('b1').rename('waterclass')
Maryland_permanent = Maryland_2018.select('waterclass').gte(75).remap([0,1],[0,2]).rename('waterclass')
Maryland_season= ee.ImageCollection([Maryland_2018.gte(25),Maryland_2018.lt(75)]).sum().eq(2)
Maryland_waterclass = ee.ImageCollection([Maryland_permanent,Maryland_season,basemap]).sum()
# 三者的交集
permanentWaterExtent = ee.ImageCollection([index_waterclass.eq(2),JRC_waterclass.eq(2),Maryland_waterclass.eq(2)]).sum().eq(3)
landExtent = ee.ImageCollection([index_waterclass.eq(0),JRC_waterclass.eq(0),Maryland_waterclass.eq(0)]).sum().eq(3)
seasonWaterExtent = ee.ImageCollection([permanentWaterExtent,landExtent]).sum().remap([0,1],[1,0]).rename('waterclass')

Map.addLayer(permanentWaterExtent.selfMask(),{'palette':['blue']},"permanent_water")
Map.addLayer(seasonWaterExtent.selfMask(),{'palette':['green']},"season water")

# 计算 AWEI在50%-100%的均值

In [8]:
def intervalmean(image):
    awei = image.updateMask(seasonWaterExtent).select('AWEI')
    intervalMean = awei.reduceRegion(**{
        'reducer': ee.Reducer.intervalMean(50,100), 
        'geometry': image.geometry(), 
        'scale': 30, 
        'bestEffort': True
    })
    return image.set({'mean': intervalMean.get('AWEI')})

In [9]:
a = sample_image.limit(34).map(intervalmean)
b = a.aggregate_array('mean')
print(b.getInfo())

[1406.723473797616, 911.1752603745903, 1690.6484589315985, 390.7014705726843, -187.41498343984307, -40.86913729659788, -1056.4397999554694, -87.45768862287066, -139.36310157163527, 181.09778741041205, 463.9730080133128, 1155.8066325614031, 1480.167056926995, 90.20146254516483, -76.38887824329858, -765.1647979966929, -613.0231663797371, -497.99747706305095, -787.5723265198249, -480.3927225853956, -729.1303805089034, -32.05357203967411, 1042.5573566850042, 480.221261298239, 398.79247891517986, 96.99033778377789, 171.45890314030146, -148.37691548501846, -229.94797914609097, 139.42574231198836, -87.9221980157181, -8.909809672148258, 426.4376253693151, 479.5116479693518]


In [12]:
number = 500
# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI']
# 分类标签
label = 'waterclass'
def imageSample(image):
    # 永久性水体
    permanent_points = image.updateMask(permanentWaterExtent).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    water_points = permanent_points.map(lambda i : i.set({'waterclass':1}))
    # 陆地
    land_points = image.updateMask(landExtent).sample(**{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    nowater_points = land_points.map(lambda i : i.set({'waterclass':0}))
    # 季节性水体
    awei = image.updateMask(seasonWaterExtent).select('AWEI').gt(ee.Image.constant(image.get('mean')))
    seasonWater = image.updateMask(awei).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    seasonWater_points = seasonWater.map(lambda i : i.set({'waterclass':1}))
    seasonLand= image.updateMask(awei.remap([0,1],[1,0])).sample(**{
        'region': roi,
        'scale': 30,
        'numPixels': number,
        'seed': 0,
        'geometries': True,
    })
    seasonLand_points = seasonLand.map(lambda i : i.set({'waterclass':0}))
    sample_points = ee.FeatureCollection([water_points,nowater_points,seasonWater_points,seasonLand_points]).flatten()
    return sample_points

In [18]:
# 总的样本点数
points_collection = a.map(imageSample).flatten()
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))
# # 水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# # 非水体的样本点数
# samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
# print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [114.67551133251911, 30.581239435333085]}, 'id': 'LC08_122039_20180111_0_33', 'properties': {'AWEI': 2734.5, 'B1': 815, 'B10': 2817, 'B11': 2808, 'B2': 851, 'B3': 1009, 'B4': 898, 'B5': 305, 'B6': 109, 'B7': 72, 'NDVI': -0.4929343308395677, 'mNDWI': 0.8050089445438283, 'ndvi_mndwi': -1.2979432753833962, 'pixel_qa': 324, 'radsat_qa': 0, 'sr_aerosol': 68, 'waterclass': 1}}
total sample number:2154


In [19]:
# 水体的样本点数
samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# 非水体的样本点数
samplepoint_water = points_collection.filter(ee.Filter.eq('waterclass',0))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

water sample number:797
water sample number:1357


In [20]:
label = 'wuhan_2018'
# csv格式
selector = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder='wuhan', file_format='csv', selectors=selector)

# shp格式
selector = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','AWEI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description=label, folder='wuhan', file_format='shp', selectors=selector)

Exporting wuhan_2014...
Exporting wuhan_2014...
